### Statistic for P116
#### the semester is devided into 2 parts
#### - from October 1st - November 20, when the Domre repair started
#### - start after dome repair: January 20th, 2026
#### - there are more files in the archive, taken before 20.01.2026, when there were still problems with the dome alignment

In [1]:
import astroquery
from astroquery.eso import Eso
import pandas as pd
import numpy as np
import sys
import math
from numpy import *
from astropy.table import Table
from astropy.table import *
from astropy.io import ascii
from datetime import datetime, date, timedelta
from termcolor import colored

eso=Eso()
eso.ROW_LIMIT = -1 


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)

Defines a general search for available data in the ESO archive
- using a date range, PIDs and instrument as input¶

In [2]:
def eso_query(instr,start,end,pid):
    num_obs= 0
    tot_exptime = 0.0
    table = eso.query_main(column_filters={'instrument':instr,'dp_cat': 'SCIENCE','stime':start,'etime':end,'prog_id':pid})
    if not table:
        num_Obs      = num_obs + 0
        tot_exptime  = 0.0
    else:
        num_Obs = num_obs + len(table)
        pd_table    = table.to_pandas()
        tot_exptime = pd_table["Exptime"].sum()
    
    return num_Obs,tot_exptime


def eso_query_target(instr,target,start,end,pid):
    num_obs= 0
    tot_exptime_ir = 0.0
    tot_exptime_ir = 0.0
    table = eso.query_main(column_filters={'instrument':instr,'target':target_,'dp_cat': 'SCIENCE','stime':start,'etime':end,'prog_id':pid})
    if not table:
        num_Obs      = num_obs + 0
        tot_exptime  = 0.0
    else:
        num_Obs = num_obs + len(table)
        pd_table    = table.to_pandas()
        tot_exptime = pd_table["Exptime"].sum()
    
    return num_Obs,tot_exptime


def eso_query_grond(instr,start,end,pid):
    num_ir = 0
    num_opt=0
    num_obs = 0
    table = eso.query_main(column_filters={'instrument': instr, 'prog_id':pid,'dp_cat': 'SCIENCE','stime':start,'etime':end} )
        
    if not table: 
        num_opt = 0
        num_nir  = 0
        tot_exptime_ir = 0.0
        tot_exptime_opt = 0.0
        tot_time = 0.0
        
    else:
# Check for optical images        
        lambda_lo       = 1100
        table_opt       = table[table['filter_lambda_min'] < lambda_lo] # limit list to only optical files
        pd_table        = table_opt.to_pandas()
        num_opt         = len(table_opt)                                     # number of optical images
        tot_exptime_opt = (pd_table["Exptime"].sum())
        table_nir       = table[table['filter_lambda_min'] > lambda_lo]      # number of near-infrared images
        pd_table        = table_nir.to_pandas()
        tot_exptime_ir  = pd_table["Exptime"].sum()                        
        num_nir = len(table_nir)
        num_obs = num_opt + num_nir
#        print(pid_1, "   ",num_nir, "    ",tot_exptime_ir)
    return num_opt,num_nir,tot_exptime_ir, tot_exptime_opt


def eso_grond_ir(instr,start,end,pid):
    num         = 0
    tot_exptime = 0
    table = eso.query_main(column_filters={'instrument': instr, 'prog_id':pid,'dp_cat': 'SCIENCE','stime':start,'etime':end} )
        
    if not table: 
        num         = 0
        tot_exptime = 0.0
        tot_time    = 0.0
        
    else:
# Check for optical images        
        lambda_lo       = 1100
        table_nir       = table[table['filter_lambda_min'] > lambda_lo]      # number of near-infrared images
        pd_table        = table_nir.to_pandas()
        tot_exptime     = pd_table["Exptime"].sum()                        
        num             = len(table_nir)


    return num,tot_exptime

def eso_grond_opt(instr,start,end,pid):
    num         = 0
    tot_exptime = 0
    table = eso.query_main(column_filters={'instrument': instr, 'prog_id':pid,'dp_cat': 'SCIENCE','stime':start,'etime':end} )
        
    if not table: 
        num         = 0
        tot_exptime = 0.0
        tot_time    = 0.0
        
    else:
# Check for optical images        
        lambda_lo       = 1100
        table_opt       = table[table['filter_lambda_min'] < lambda_lo]      # number of near-infrared images
        pd_table        = table_opt.to_pandas()
        tot_exptime     = pd_table["Exptime"].sum()                        
        num = len(table_opt)

    return num,tot_exptime





#### Total exposure times for GROND are calculated based on number of IRACE exposures and IRACE exposure times¶

In [5]:
info = "/home/angela/LaSilla/P116/P116_pi.list"

data = open(info,'r')

#start   = input('Start date of query (yyyy-mm-dd):  ')
#end     = input('End date of query (yyyy-mm-dd):    ')
start_1 = "2025-10-01"
end_1   = "2025-11-21"
start_2 = "2026-01-20"
end_2 = (datetime.now()+timedelta(days=2)).strftime('%Y-%m-%d')             # date for tomorrow, to make sure that all nights are  used
#end = "2025-05-28"


text = colored(("Dome repair between: ",end_1, "and :", start_2), "red")

print("Statistics until: ",text, "to make sure all data are found")   

end_f   = (datetime.now() + timedelta(days=-1) ).strftime('%Y-%m-%d')     # date for yesterday, the day the last observing night started
#end_f   = end.strftime('%Y-%m-%d')

print("     ")
print("     ")
print("     ")

orig_stdout = sys.stdout

path = "/home/angela/LaSilla/P116"

file_out   = path + "/Statistic/P116_stats.cvs"
file_out_2 = path + "/Statistic/P116_stats_a.cvs"

data_2 = []
col1=[]    # full name
col2=[]    # list of PIDs
col3=[]    # instrument
col4=[]    # allocated time in hours Category A
col5=[]    #  requested time in proposal
col6=[]    # number of files
col7=[]    # exposure times
col8=[]    # execution time in hr
col9=[]    # fraction in %
col10 =[]  # fraction completed in of total %


data_3 = []
cola=[]    # full name
colb=[]    # 1st PID
colc=[]    # 2nd PID
cold=[]    # instrument
cole=[]    # allocated time A in hr
colf=[]    # tie asked for in proposal
colg=[]    # number of files
colh=[]    # exposure time in hr
coli=[]    # observed time in hr
colj=[]    # fraction completed in %
colk=[]    # fraction of total time asked for in proposal

num_DDT     = 0
num         = 0
tot_exptime = 0.0
header1 = data.readline()
header2 = data.readline()

for line in data:    
    num = 0
    tot_exptime = 0.0
    tot_time    = 0.0
    
    line     = line.strip()
    columns  = line.split()
    name     = columns[0]
    first    = columns[1]
    email    = columns[2]
    pid_1    = columns[3]
    pid_2    = columns[4]
    instr    = columns[5]
    t_tot_A  = float(columns[6])    
    t_tot_B  = float(columns[7])
    tot_alloc_time = t_tot_A  + t_tot_B 
#    print(name,tot_alloc_time)
    f_name = name+','+ first
    full_name = ("{:24}".format(f_name))

    idlist = pid_1
    if pid_2 != str("000.0000.000"):
        idlist = idlist+", "+pid_2


    
#############################################################################################
   
# Check FEROS programs
  
    if instr  == "FEROS":
        num         = 0
        tot_exptime = 0.0
        tot_time    = 0.0
# check before dome repair        
        result_1_a = eso_query(instr,start_1,end_1,pid_1)
        num = num+result_1_a[0]
        tot_exptime= tot_exptime+result_1_a[1]
        result_1_b = eso_query(instr,start_1,end_1,pid_2)
        num = num+result_1_b[0]
        tot_exptime= tot_exptime+result_1_b[1]

# check after dome repair        
        result_2_a = eso_query(instr,start_2,end_2,pid_1)
        num= num + result_2_a[0]
        tot_exptime = tot_exptime+result_2_a[1]

        result_2_b = eso_query(instr,start_2,end_2,pid_2)
        num= num + result_2_b[0]
        tot_exptime = tot_exptime+result_2_b[1]
        
        oh_time = 282.0
        tot_exptime = tot_exptime/3600.0
        tot_time    = ((num*oh_time)/3600.0)  + tot_exptime

        if name== "KORHONEN":                            # usual does 3 exposures in 1 OB, overheads are 06:06 min=366 sec         
            num           = 0
            tot_exptime   = 0.0
            tot_time_     = 0.0          
            oh_time       = 55.0                        # mean overhead
            result_1_a   = eso_query(instr,start_1,end_1,pid_1)
            num          = num+result_1_a[0]
#            print("Number of files bevore dome repair: ", num     )
            tot_exptime  = tot_exptime+result_1_a[1]

# Check after dome repair was finishe
            result_1_b   = eso_query(instr,start_2,end_2,pid_1)
            num          = num+result_1_b[0]
            tot_exptime  = (tot_exptime+result_1_b[1])/3600.0

            tot_time     = ((num*oh_time)/3600.0)  + tot_exptime
#            print("Total number of files: ", num     )              
      


###########################################################################################       
    #    Check WFI programs
    if instr == "WFI" :
        tot_time= 0.0
        if name =="BANADOS":
            oh_time      = 226.0            # new overhead, 3.5 min for fetch... + setup + readout
            num          = 0
            tot_time     = 0.0
            tot_exptime  = 0.0

            result_1_a   = eso_query(instr,start_1,end_1,pid_1)
            num          = num+result_1_a[0]
            tot_exptime  = tot_exptime+result_1_a[1]
            result_1_b   = eso_query(instr,start_1,end_1,pid_2)
            num          = num+result_1_b[0]
#            print("Number of files bevore dome repair: ", num     )
            tot_exptime  = tot_exptime+result_1_a[1]+ result_1_a[1]

# Check after dome repair was finishe
            result_2_a   = eso_query(instr,start_2,end_2,pid_1)
            num          = num+result_2_a[0]
            tot_exptime  = tot_exptime+result_2_a[1]
            result_2_b   = eso_query(instr,start_2,end_2,pid_2)
            num          = num+result_2_b[0]
            tot_exptime  = (tot_exptime+result_2_b[1])/3600.0
            tot_time     = ((num*oh_time)/3600.0)  + tot_exptime          
        
        if name=="RAU":
            num          = 0
            tot_exptime  = 0.0
            tot_time     = 0.0
            overheads    =230.0                                       # per 4 images
            result_1_a   = eso_query(instr,start_1,end_1,pid_1)
            num          = num+result_1_a[0]
            tot_exptime  = tot_exptime+result_1_a[1]

# Check after dome repair was finished
            result_1_b   = eso_query(instr,start_2,end_2,pid_1)
            num          = num+result_1_b[0]
            tot_exptime  = tot_exptime+result_1_b[1]

            #Check the second PID 
            result_2_a   = eso_query(instr,start_1,end_1,pid_2)
            num          = num+result_2_a[0]
            tot_exptime  = tot_exptime+result_2_a[1]
            result_2_b   = eso_query(instr,start_2,end_2,pid_2)
            num          = num+result_2_b[0]
            tot_exptime  = (tot_exptime+result_2_b[1])/3600.0 
            
            tot_time     = ((num/4*oh_time)/3600.0)  + tot_exptime

        
        if name == "SUYU":
            num          = 0
            tot_exptime  = 0.0
            tot_time     = 0.0
            oh_time      = 610.0                                       # per 4 images
            result_1_a   = eso_query(instr,start_1,end_1,pid_1)
            num          = num+result_1_a[0]
            tot_exptime  = tot_exptime+result_1_a[1]

# Check after dome repair was finished
            result_1_b   = eso_query(instr,start_2,end_2,pid_1)
            num          = num+result_1_b[0]
            tot_exptime  = tot_exptime+result_1_b[1]

            #Check the second PID 
            result_2_a   = eso_query(instr,start_1,end_1,pid_2)
            num          = num+result_2_a[0]
            tot_exptime  = tot_exptime+result_2_a[1]
            result_2_b   = eso_query(instr,start_2,end_2,pid_2)
            num          = num+result_2_b[0]
            tot_exptime  = (tot_exptime+result_2_b[1] )/3600.0
            
            tot_time     = ((num/4*oh_time)/3600.0)  + tot_exptime

        if name == "GREDEL" or name == "STANKE":  
            num          = 0
            tot_exptime  = 0.0
            tot_time     = 0.0
            overheads    = 610.0                                       # per 4 images
            result_1_a   = eso_query(instr,start_1,end_1,pid_1)
            num          = num+result_1_a[0]
            tot_exptime  = tot_exptime+result_1_a[1]

# Check after dome repair was finished
            result_1_b   = eso_query(instr,start_2,end_2,pid_1)
            num          = num+result_1_b[0]
            tot_exptime  = tot_exptime+result_1_b[1]

            #Check the second PID 
            result_2_a   = eso_query(instr,start_1,end_1,pid_2)
            num          = num+result_2_a[0]
            tot_exptime  = tot_exptime+result_2_a[1]
            result_2_b   = eso_query(instr,start_2,end_2,pid_2)
            num          = num+result_2_b[0]
            tot_exptime  = tot_exptime+result_2_b[1] /3600.0
            
            tot_time     = ((num/4*oh_time) /3600) + tot_exptime

            
 


                
###################################################################################
# Check  GROND   programms
# output of eso_query_grond: num_opt,num_nir,tot_exptime_ir, tot_exptime_opt
    
    if instr == "GROND":
        num_opt       = 0
        num_ir        = 0
        overhead      = 220.0                                              # overheads per OB are now 3.5 min, 210 sec
        tot_exptime   = 0.0                                                # exposure time in optical
        tot_time      = 0.0                                                # execution time#
        
            
        if name=="RAU":
            num_opt       = 0
            num_ir        = 0
            overhead      = 220.0                                              # overheads per OB are now 3.5 min, 210 sec
            tot_exptime   = 0.0                                                # exposure time in optical
            tot_time      = 0.0 
 # check before dome repair            
            result_opt_1  = eso_grond_opt(instr,start_1,end_1,pid_1)
            num_opt       = num_opt + result_opt_1[0] # number of opt images, optical is not used
            tot_exptime   = tot_exptime + result_opt_1[1]
            result_opt_2  = eso_grond_opt(instr,start_1,end_1,pid_2)
            num_opt       = num_opt + result_opt_2[0] 
            tot_exptime   = (tot_exptime +result_opt_2[1]) 
            result_ir_1   = eso_grond_ir(instr,start_1,end_1,pid_1)
            num_ir        = num_ir + result_ir_1[0]
            result_ir_2   = eso_grond_ir(instr,start_1,end_1,pid_2)
            num_ir        = num_ir + result_ir_2[0]
#    
# check after dome repair   
            result_opt_3  = eso_grond_opt(instr,start_2,end_2,pid_1)
            num_opt       = num_opt + result_opt_3[0]                   # number of opt images
            result_opt_4  = eso_grond_opt(instr,start_2,end_2,pid_2)
            num_opt       = num_opt + result_opt_4[0] 
            result_ir_3   = eso_grond_ir(instr,start_2,end_2,pid_1)
            num_ir        = num_ir + result_ir_3[0]
            result_ir_4   = eso_grond_ir(instr,start_2,end_2,pid_2)
            num_ir        = num_ir + result_ir_4[0] 
            
            tot_exptime   = (tot_exptime + result_opt_3[1]+result_opt_4[1])/3600.0            
            num           = num_opt +num_ir    

            tot_time    = (((num_opt/6)*overhead)/3600.0) + tot_exptime
       
        if name=="SUYU":
            num_opt       = 0
            num_ir        = 0
            overhead      = 220.0                                              # overheads per OB are now 3.5 min, 210 sec
            tot_exptime   = 0.0                                                # exposure time in optical
            tot_time      = 0.0 
 # check before dome repair            
            result_opt_1  = eso_grond_opt(instr,start_1,end_1,pid_1)
            num_opt       = num_opt + result_opt_1[0] # number of opt images, optical is not used
            tot_exptime   = tot_exptime + result_opt_1[1]
            result_opt_2  = eso_grond_opt(instr,start_1,end_1,pid_2)
            num_opt       = num_opt + result_opt_2[0] 
            tot_exptime   = (tot_exptime +result_opt_2[1]) 
            result_ir_1   = eso_grond_ir(instr,start_1,end_1,pid_1)
            num_ir        = num_ir + result_ir_1[0]
            result_ir_2   = eso_grond_ir(instr,start_1,end_1,pid_2)
            num_ir        = num_ir + result_ir_2[0]
#    
# check after dome repair   
            result_opt_3  = eso_grond_opt(instr,start_2,end_2,pid_1)
            num_opt       = num_opt + result_opt_3[0]                   # number of opt images
            result_opt_4  = eso_grond_opt(instr,start_2,end_2,pid_2)
            num_opt       = num_opt + result_opt_4[0] 
            result_ir_3   = eso_grond_ir(instr,start_2,end_2,pid_1)
            num_ir        = num_ir + result_ir_3[0]
            result_ir_4   = eso_grond_ir(instr,start_2,end_2,pid_2)
            num_ir        = num_ir + result_ir_4[0] 
            
            tot_exptime   = (tot_exptime + result_opt_3[1]+result_opt_4[1])/3600.0            
            num           = num_opt +num_ir    

            tot_time    = (((num_opt/6)*overhead)/3600.0) + tot_exptime
       
        if name=="BANADOS":
            num_opt       = 0
            num_ir        = 0
            overhead      = 220.0                                              # overheads per OB are now 3.5 min, 210 sec
            tot_exptime   = 0.0                                                # exposure time in optical
            tot_time      = 0.0 
 # check before dome repair            
            result_opt_1  = eso_grond_opt(instr,start_1,end_1,pid_1)
            num_opt       = num_opt + result_opt_1[0] # number of opt images, optical is not used
            tot_exptime   = tot_exptime + result_opt_1[1]
            result_opt_2  = eso_grond_opt(instr,start_1,end_1,pid_2)
            num_opt       = num_opt + result_opt_2[0] 
            tot_exptime   = (tot_exptime +result_opt_2[1]) 
            result_ir_1   = eso_grond_ir(instr,start_1,end_1,pid_1)
            num_ir        = num_ir + result_ir_1[0]
            result_ir_2   = eso_grond_ir(instr,start_1,end_1,pid_2)
            num_ir        = num_ir + result_ir_2[0]
#    
# check after dome repair   
            result_opt_3  = eso_grond_opt(instr,start_2,end_2,pid_1)
            num_opt       = num_opt + result_opt_3[0]                   # number of opt images
            result_opt_4  = eso_grond_opt(instr,start_2,end_2,pid_2)
            num_opt       = num_opt + result_opt_4[0] 
            result_ir_3   = eso_grond_ir(instr,start_2,end_2,pid_1)
            num_ir        = num_ir + result_ir_3[0]
            result_ir_4   = eso_grond_ir(instr,start_2,end_2,pid_2)
            num_ir        = num_ir + result_ir_4[0] 

            tot_exptime   = (tot_exptime + result_opt_3[1]+result_opt_4[1])/3600.0            
            num           = num_opt +num_ir    

            tot_time    = (((num_opt/6)*overhead)/3600.0) + tot_exptime

        if name=="WANG":
            num_opt       = 0
            num_ir        = 0
            overhead      = 220.0                                              # overheads per OB are now 3.5 min, 210 sec
            tot_exptime   = 0.0                                                # exposure time in optical
            tot_time      = 0.0 
 # check before dome repair            
            result_opt_1  = eso_grond_opt(instr,start_1,end_1,pid_1)
            num_opt       = num_opt + result_opt_1[0] # number of opt images, optical is not used
            tot_exptime   = tot_exptime + result_opt_1[1]
            result_opt_2  = eso_grond_opt(instr,start_1,end_1,pid_2)
            num_opt       = num_opt + result_opt_2[0] 
            tot_exptime   = (tot_exptime +result_opt_2[1]) 
            result_ir_1   = eso_grond_ir(instr,start_2,end_1,pid_1)
            num_ir        = num_ir + result_ir_1[0]
            result_ir_2   = eso_grond_ir(instr,start_1,end_1,pid_2)
            num_ir        = num_ir + result_ir_2[0]
#    
# check after dome repair   
            result_opt_3  = eso_grond_opt(instr,start_2,end_2,pid_1)
            num_opt       = num_opt + result_opt_3[0]                   # number of opt images
            result_opt_4  = eso_grond_opt(instr,start_2,end_2,pid_2)
            num_opt       = num_opt + result_opt_4[0] 
            result_ir_3   = eso_grond_ir(instr,start_2,end_2,pid_1)
            num_ir        = num_ir + result_ir_3[0]
            result_ir_4   = eso_grond_ir(instr,start_2,end_2,pid_2)
            num_ir        = num_ir + result_ir_4[0] 

            tot_exptime   = (tot_exptime + result_opt_3[1]+result_opt_4[1])/3600.0            
            num           = num_opt +num_ir    

            tot_time    = (((num_opt/6)*overhead)/3600.0) + tot_exptime
            
    
#### Calculate percentages of allocated times

    time_perc_A = 0.0                                                       # percentage of Cat A. time observed
    time_perc_B = 0.0                                                       # percentage of Cat B. time observed
    
# Case 1: only category A is assigned, and total execution time is greater than 0.0:
    
    if tot_time > 0.0 and t_tot_A > 0.0 and t_tot_B == 0.0:
        time_perc_A = (100.0* (tot_time /t_tot_A))
        time_perc_B = 0.0
    elif tot_time > 0.0 and t_tot_A > 0.0 and t_tot_B > 0.0 and tot_time < t_tot_A:   # time observed is less than Cat. A
        time_perc_A = (100.0* (tot_time /t_tot_A))
        

# Case 2: only category B is assigned, and total execution time is greater than 0.0:        
    elif  tot_time > 0.0 and t_tot_B > 0.0 and t_tot_A == 0.0:  
        time_perc_B = 100.0* (tot_time /t_tot_B)
        time_perc_A = 0.0

# Case 3: both category A & B are assigned, and the total execution time is greater than the time assigned to Category A:    
    elif tot_time > t_tot_A and t_tot_B > 0.0  and t_tot_A !=0: 
        time_perc_A = 100.0
        time_perc_B = (100.0* (tot_time /t_tot_A))-100.0

# Case 4: no time has been assigned to either category (not useful):    
    elif t_tot_A == 0 and t_tot_B == 0:
        time_perc_A = 0.0
        time_perc_B = 0.0

# Case 5: no observation has been done:
    elif tot_exptime  == 0.0:
        time_perc_A   = 0.0
        time_perc_B   = 0.0




### Format the values
    
    t_tot_A_f      = ("{:>6.0f}".format(t_tot_A))                            # formatted allocated time cat- A
    t_tot_B_f      = ("{:>6.0f}".format(t_tot_B))                            # formatted allocated time cat- B
    tot_time_f     = ("{:>6.2f}".format(tot_time))                           # formatted observed time
    time_perc_A_f  = ("{:>6.2f}".format(time_perc_A))
    time_perc_B_f  = ("{:>6.2f}".format(time_perc_B))
    tot_exptime_f  = ("{:>6.2f}".format(tot_exptime))
#    print(name,num, tot_time_f,tot_exptime_f)

    data_2.append({'#PI_name                ': full_name.ljust(20," "),
                   'PIDs              ': idlist.ljust(30," "),
                   'Inst.': str(instr).center(5," "),
                   'Cat-A [h]': str(t_tot_A_f).rjust(10," "),
                   'Cat-B [h]': str(t_tot_B_f).rjust(10," "),
                   '# files': str(num).rjust(10," "),
                   'exp. [h]': tot_exptime_f,
                   'exec. [h] ': str(tot_time_f).center(10," "),
                   '  A [%]': time_perc_A_f,
                   '  B [%]': time_perc_B_f
        }              
    )
    col1.append(full_name)
    col2.append(idlist.ljust(30," "))
    col3.append(instr.center(9," "))
    col4.append(t_tot_A_f)
    col5.append(t_tot_B_f)
    col6.append(num)
    col7.append(tot_exptime_f)
    col8.append(tot_time_f)
    col9.append(time_perc_A_f)
    col10.append(time_perc_B_f)
              
    data_3.append({'#PI_name': full_name.ljust(20," "),
                   '       PID_1       ': pid_1.center(15," "),
                   '       PID_2       ': pid_2.center(15," "),
                   'Instrument  ': str(instr).center(5," "),
                   'Cat-A [h]': str(t_tot_A_f).rjust(10," "),
                   'Cat-B [h]': str(t_tot_B_f).rjust(10," "),
                   '# of files': str(num).rjust(10," "),
                   'exp. [h]': tot_exptime_f,
                   'exec.[h] ': str(tot_time_f).center(10," "),
                   '  A [%]': time_perc_A_f,
                   '  B.[%]': time_perc_B_f
        }              
    )
    cola.append(full_name)
    colb.append(pid_1.center(15," "))
    colc.append(pid_2.center(15," "))
    cold.append(instr.center(15," "))
    cole.append(t_tot_A_f)
    colf.append(t_tot_B_f)
    colg.append(num)
    colh.append(tot_exptime_f)
    coli.append(tot_time_f)
    colj.append(time_perc_A_f)
    colk.append(time_perc_B_f)


pd.set_option('display.precision', 1)    
df = pd.DataFrame(data_2)
df_1 = pd.DataFrame(data_3)
number = len(df.index)
#print(number)
print(df.to_string(index=False))
#print(new)
li = [df.columns.values.tolist()] + df.values.tolist()
df.to_csv(file_out, quoting=None,index=False) 
df_1.to_csv(file_out_2, index=False) 
print("    ")
print("    ")

print("Finished")
sys.stdout=orig_stdout

Statistics until:  ('Dome repair between: ', '2025-11-21', 'and :', '2026-01-20') to make sure all data are found
     
     
     
 #PI_name                             PIDs               Inst.  Cat-A [h]  Cat-B [h]    # files exp. [h] exec. [h]    A [%]   B [%]
 RAU,Arne                 116.29GN.001, 114.27S7.001     GROND        117          0       1663     9.80    11.77     10.06    0.00
 WANG,Lingzhi             116.29GJ.001                   GROND         21          0       1841     4.25     6.77     32.24    0.00
 SUYU,Sherry              116.29GC.003, 115.28JZ.002     GROND        117          0       1105    11.04    12.65     10.81    0.00
 BOSMAN,Sarah             116.29G8.002                   GROND          0         16          0     0.00     0.00      0.00    0.00
 BANADOS,Eduardo          116.29G8.001                   GROND         42          0       3333    14.02    18.54     44.14    0.00
 RAU,Arne                 116.29GQ.001, 116.29GR.001     FEROS          0   

In [22]:
import reportlab
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter,A4
from reportlab.lib.units import inch
from reportlab.lib import colors
import os
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle,Paragraph,Spacer
from reportlab.lib.styles import ParagraphStyle,getSampleStyleSheet
from reportlab.lib.units import cm

styles = getSampleStyleSheet()

path = "/home/angela/LaSilla/P116/Statistic/"
date_now = (datetime.now()).strftime('%Y-%m-%d')
pdf_out = path + end_f+".pdf"
print(pdf_out)
doc = SimpleDocTemplate(pdf_out, pagesize=A4,fontsize=8)
# container for the 'Flowable' objects
elements = []



colw = (1.3*inch,1.2*inch,0.55*inch,0.6*inch, 0.6*inch, 0.6*inch, 0.6*inch,0.6*inch, 0.6*inch,0.6*inch)
t=Table(li,colw,(number+1)*[0.3*inch])

t.setStyle(TableStyle([('ALIGN',(1,1),(8,number),'CENTER'),
                       ('ALIGN',(1,1),(1,number),'LEFT'),
                       ('FONTSIZE',(0,0),(10,number),5),
                       ('TEXTCOLOR',(1,1),(10,number),colors.black),
                       ('TEXTCOLOR',(0,0),(0,-1),colors.blue),
                       ('BACKGROUND', (0, 0), (0, number), colors.lightblue),
                       ('BACKGROUND', (1, 0), (10, 0), colors.lightblue),
                       ('BACKGROUND',(0,number-4),(0,number),colors.orange),
                       ('VALIGN',(0,-1),(-1,-1),'MIDDLE'),
                       ('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),
                       ('BOX', (0,0), (-1,-1), 0.25, colors.black),
                       ]))
elements.append(t)


title = "P116 from " + start + " until "+end_f

flowables = [
    Paragraph(title, styles['Title']),
    t,
    Spacer(1 * cm, 1 * cm),
Paragraph('- Dome repair: 20.10.2025 - 10.01.2026'),    
Paragraph('- Chile time is given in number of nights NOT hours, 1 night= 7 hours!!!'),

]
doc.build(flowables)

/home/angela/LaSilla/P116/Statistic/2026-01-11.pdf


### If you want to test a single PID for WFI

In [48]:
########WFI
pid_1        = "116.29G7.001"            # 
pid_2        = "000.0000.000"
instr        = "WFI"
start_1      = "2025-10-01"
end_1        = "2025-11-21"              # Dome repair starts
start_2      = "2026-01-20"              # first usefull data taken
end_2        = (datetime.now()+timedelta(days=2)).strftime('%Y-%m-%d') 

oh_time      = 230.0            # new overhead, 3.5 min for fetch... + setup + readout
num          = 0
tot_time     = 0.0
tot_exptime  = 0.0

result_1_a   = eso_query(instr,start_1,end_1,pid_1)
num          = num+result_1_a[0]
tot_exptime  = tot_exptime+result_1_a[1]
result_1_b   = eso_query(instr,start_1,end_1,pid_2)
num          = num+result_1_b[0]
print("Number of files bevore dome repair: ", num     )
tot_exptime  = tot_exptime+result_1_a[1]+ result_1_a[1]

# Check after dome repair was finishe
result_2_a   = eso_query(instr,start_2,end_2,pid_1)
num          = num+result_2_a[0]
tot_exptime  = tot_exptime+result_2_a[1]
result_2_b   = eso_query(instr,start_2,end_2,pid_2)
num          = num+result_2_b[0]
tot_exptime  = (tot_exptime+result_2_b[1]) / 3600.0
tot_time     = ((num*oh_time)/3600.0)  + tot_exptime

print("Total number of files: ", num     )
print("Total exposure time :", tot_exptime)



Number of files bevore dome repair:  0
Total number of files:  12
Total exposure time : 0.17916777777777781


### or FEROS

In [4]:
pid_1        = "116.29G9.001"            #
instr        = "FEROS"
start_1      = "2025-10-01"
end_1        = "2025-11-21"              # Dome repair starts
start_2      = "2026-01-20"              # first usefull data taken
end_2        = (datetime.now()+timedelta(days=2)).strftime('%Y-%m-%d') 

num          = 0
tot_exptime  = 0.0
tot_time     = 0.0
oh_time      = 55.0


result_1_a   = eso_query(instr,start_1,end_1,pid_1)
num          = num+result_1_a[0]
print("Number of files bevore dome repair: ", num     )
tot_exptime  = tot_exptime+result_1_a[1]

# Check after dome repair was finishe
result_1_b   = eso_query(instr,start_2,end_2,pid_1)
num          = num+result_1_b[0]
tot_exptime  = tot_exptime+result_1_b[1]

tot_time     = ((num*oh_time)  + tot_exptime) / 3600.0
print("Total number of files: ", num     )
print("Total exposure time (h):", tot_exptime/3600.0)

Number of files bevore dome repair:  40
Total number of files:  40
Total exposure time (h): 20.97834333333333


#### and GROND, e.g. Bañados, Rau or Suyu

In [4]:
pid_1        = "116.29GJ.001"            #  Wang
pid_2        = "000.0000.000"            #  empty
instr        = "GROND"
start_1      = "2025-10-01"
end_1        = "2025-11-22"              # Dome repair starts
start_2      = "2026-01-20"              # first usefull data taken
end_2        = (datetime.now()+timedelta(days=2)).strftime('%Y-%m-%d') 

num          = 0
num_opt      = 0
num_ir       = 0
tot_exptime  = 0.0
tot_time     = 0.0
overhead= 282.0
name         = "WANG"
if name     == "WANG":
        
# check before dome repair            
    result_opt_1  = eso_grond_opt(instr,start_1,end_1,pid_1)
    num_opt       = num_opt + result_opt_1[0] # number of opt images, optical is not used
    tot_exptime   = tot_exptime + result_opt_1[1]
    result_opt_2  = eso_grond_opt(instr,start_1,end_1,pid_2)
    num_opt       = num_opt + result_opt_2[0] 
    tot_exptime   = (tot_exptime +result_opt_2[1]) 

    print("Number of opt. files bevore dome repair: ", num_opt )    
    print("Total optical exposure time before dome repair (hours): ", tot_exptime /3600.0)
    
    result_ir_1   = eso_grond_ir(instr,start_1,end_1,pid_1)
    num_ir        = num_ir + result_ir_1[0]
    result_ir_2   = eso_grond_ir(instr,start_1,end_1,pid_2)
    num_ir        = num_ir + result_ir_2[0]


    
    print("Number of nir files before dome repair: ", num_ir )   
    print("              OK                     ")
    # check after dome repair   
    print(start_2,end_2)
    result_opt_3  = eso_grond_opt(instr,start_2,end_2,pid_1)
    num_opt       = num_opt + result_opt_3[0]                   # number of opt images
    result_opt_4  = eso_grond_opt(instr,start_2,end_2,pid_2)
    num_opt       = num_opt + result_opt_4[0] 
    result_ir_3   = eso_grond_ir(instr,start_2,end_2,pid_1)
    num_ir        = num_ir + result_ir_3[0]
    result_ir_4   = eso_grond_ir(instr,start_2,end_2,pid_2)
    num_ir        = num_ir + result_ir_4[0] 
            
    tot_exptime   = (tot_exptime + result_opt_3[1]+result_opt_4[1])/3600.0            
    num           = num_opt +num_ir    

    tot_time    = (((num_opt/6)*overhead)/3600.0) + tot_exptime
       

print("Total number of opt. images for 116: ",num_opt)
print("Total number of ir images for 116: ",num_ir)
print("Total execution time (hours): ", tot_time)
print("Total number of files:",num)

Number of opt. files bevore dome repair:  0
Total optical exposure time before dome repair (hours):  0.0
Number of nir files before dome repair:  0
              OK                     
2026-01-20 2026-02-14
Total number of opt. images for 116:  247
Total number of ir images for 116:  1594
Total execution time (hours):  7.478930277777778
Total number of files: 1841
